In [ ]:
import numpy as np # math
from scipy.interpolate import interp1d # interpolation 1d
from scipy.stats import norm # normal distribution
import matplotlib.pyplot as plt # graphs and plots
import pandas as pd
import datetime
import matplotlib.dates as mdates

In [ ]:
# import of the excel with the show up profiles
show_up_ter = pd.read_excel(
    r"/home/antoine/projects/KAPpaxsim/KAPpaxsim/data/raw/ADRM_param_full.xlsx",
    sheet_name=r"terminal",
    header=1,
)

show_up_ter = show_up_ter.drop([0, 1], axis=0)
show_up_ter = show_up_ter.reset_index(drop=True)

In [ ]:
show_up_ter.head()

In [ ]:
# interpolation of show_up profiles and inverse functions
x = show_up_ter["time before STD"].to_numpy(dtype=float)

In [ ]:
flight_types = ['FSC',"LCC","EARLY","CHINA"]

dct_y = {
    flight_type : show_up_ter["cumulative distribution {}".format(flight_type)] for flight_type in flight_types
}

In [ ]:
dct_f_ter = {
    flight_type : interp1d(x, dct_y[flight_type], kind="linear") for flight_type in flight_types
}

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
x = np.linspace(0, 360, 100)

for key in dct_f_ter:
    ax.plot(x, dct_f_ter[key](x), label="show-up {}".format(key))

ax.invert_xaxis()

ax.legend()
# ax.set(xlim=(0, 360), ylim=(0, 1))
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))

ax.plot([0,1,2,3,4],[0,2,4,6,8], label="double_yokhr")
ax.plot([0,1,2,3,4],[0,1,2,3,4], label="yokhr",color="red")


plt.legend()

In [ ]:
dct_f_ter

In [ ]:
dct_f_ter_inv = {flight_type : interp1d(dct_f_ter[flight_type](x), x, kind="linear") for flight_type in flight_types }

In [ ]:
from src.utils.profiles_from_schedule_new import show_up_generator, Schedule # class for show-up generator
test = Schedule()
path_to_6k_sched = r"/home/antoine/projects/KAPpaxsim/KAPpaxsim/data/processed/Schedule (30th terminal peak, 6000 pax)_PROCESSED.xlsx"
test.from_path(path=path_to_6k_sched)
test.cleanup()
test.filter()

In [ ]:
# only FSC flights
# our schedule is stored in test.data

# we need a column "show_up_category" in the schedule dataframe
list_time_Pax = []
list_flights = []
list_ST = []
list_category = []

# I want to create a table like:
#     "Flight Number"
#     "time": pax show-up time
#     "Scheduled Time": flight STD
#     "Category": type (eg. early, china. fsc, LCC)

In [ ]:
test.schedule.head(n=1)

In [ ]:
test.schedule['PAX_SUM FC'].replace('-',0,inplace=True)

In [ ]:
# important: boolean indexing on DataFrames
mask_LCC = test.airline_code["FSC / LCC"] == "LCC"
test.airline_code[mask_LCC]

In [ ]:
test.data.head(n=2)

In [ ]:
# let's define flight categories
# first check if early
# then check if china
# then check if LCC
# else, its FSC

# first, let's prepare lists of LCC
list_LCC = test.airline_code[mask_LCC]["airline code"].to_numpy(dtype="str")

# loop through schedule and test with if condtions

for i in range(len(test.data)):
    std =test.data.loc[i, "Scheduled Time"]
    if std < pd.to_datetime(
                "2020-10-13 08:00:00"
            ) and std >= pd.to_datetime(
                "2020-10-13 02:00:00"
            ):
        category = "EARLY"

    elif test.data.loc[i, "Intl Regions"] == "China":
        category = "China"

    elif test.data.loc[i, "Flight Number"][0:2] in list_LCC:
        category = 'LCC'

    else:
        category = "FSC"

    test.data.loc[i,"show_up_category"] = category



In [ ]:

# loop trhough each flight (= line of flight schedule)
for i in range(len(test.data)):
    N_flight_pax = int(test.data.loc[i, "PAX_SUM FC"])
    std = test.data.loc[i, "Scheduled Time"]
    show_up_category = test.data.loc[i,"show_up_category"]
    flight_number = test.data.loc[i, "Flight Number"]

    y = np.linspace(0.0001, 0.995, N_flight_pax)
    f_ter_inv_linear = dct_f_ter_inv[show_up_category]

    time_Terminal = ( 
        std.hour * 60
        + std.minute
        - f_ter_inv_linear(y)
    )

    for t in time_Terminal:
        t = datetime.datetime(
            year=2020,
            month=10,
            day=13,
            hour=int((t % (24 * 60)) / 60),
            minute=int(t % 60),
            second=int(t % 1 * 60),
        )
        list_time_Pax.append(t)
        list_flights.append(flight_number)
        list_ST.append(std)
        list_category.append(show_up_category)


In [ ]:

dct_Pax = {
    "Flight Number": list_flights,
    "time": list_time_Pax,
    "Scheduled Time": list_ST,
    "Category": list_category,
}
df_Pax = pd.DataFrame(dct_Pax)

In [ ]:
df_Pax['Pax']=1

In [ ]:
plot = (
    df_Pax.set_index("time", drop=False)["Pax"]
    .resample("5min")
    .agg(["sum"])
    .rolling(window=12, center=True)
    .mean()
    .dropna()
)

fig, ax = plt.subplots(figsize=(8, 5))
ax.plot(plot * 12, label="total show-up")

# plot param
xmin = pd.to_datetime("2020-10-13 00:00:00")
xmax = pd.to_datetime("2020-10-14 00:00:00")
plt.rcParams.update({"figure.autolayout": True})
hours = mdates.HourLocator(interval=1)
half_hours = mdates.MinuteLocator(byminute=[0, 30], interval=1)
h_fmt = mdates.DateFormatter("%H:%M")

# formatting
ax.set_xlim((xmin, xmax))
ax.set_xticks(plot.index.to_list())
ax.set_xticklabels(ax.get_xticks(), rotation=45, **{"horizontalalignment": "right"})
ax.xaxis.set_major_locator(hours)
ax.xaxis.set_major_formatter(h_fmt)
ax.xaxis.set_minor_locator(half_hours)
ax.legend(loc="upper left", frameon=False)

plt.show()
